In [26]:
library(INLA)
library(rSPDE)
library(readr)
library(fmesher)
library(inlabru)
if (FALSE){
    data_ori <- as.data.frame(read_csv("tmp/1180_vol.csv"))
    logpt<-log(data_ori$Close)
    logreturn <-logpt[2:length(logpt)]-logpt[1:(length(logpt)-1)]
    date <- data_ori$Date[1:(length(data_ori$Date)-1)]
    

}else{
  data_ori <- as.data.frame(read_csv("tmp/IF_logreturn.csv"))
  logreturn <-data_ori$logreturn
  data_ori$Date<-data_ori[,1]
  date <- data_ori$Date

}
date_timeindex<-as.numeric(as.POSIXct(date)) 
date_timeindex <- (date_timeindex - min(date_timeindex))/86400
mesh <- fm_mesh_1d(date_timeindex)
data = list(logreturn = logreturn, times = date_timeindex)





New names:
• `Date` -> `Date...1`
• `Date` -> `Date...3`
Rows: 1170 Columns: 6
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (1): ins_id
dbl  (4): Date...3, close, logreturn, return
date (1): Date...1

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [28]:
build_index <- function(data_size, window,size=10) {
    train <- test <- list()

    k = 1
    by=as.integer(length(data$times)/size)
    by=max(by,window)
    for (i in seq(from = window, to = data_size - window, by = by)) {
        train[[k]] <- 1:i
        test[[k]] <- (i + 1):(i + window)
        k = k + 1
    }
    index = list(train = train, test = test)
    return(index)
}


In [29]:
exp_id=31
library(glue)
config_id<-11
run_id=0
fit_path=glue("/home/jianj0c/project/STP/notebook_SPDE/stochasticVolatility/Log/{exp_id}.fitting/fits")
results<-readRDS(file.path(fit_path, glue("fit_{config_id}.rds")))
rough_stochvol<-results$fit[[run_id+1]]$fit

config_id<-71
run_id=0
fit_path=glue("/home/jianj0c/project/STP/notebook_SPDE/stochasticVolatility/Log/{exp_id}.fitting/fits")
results<-readRDS(file.path(fit_path, glue("fit_{config_id}.rds")))
rough_stochvol.t<-results$fit[[run_id+1]]$fit

config_id<-47
run_id=0
fit_path=glue("/home/jianj0c/project/STP/notebook_SPDE/stochasticVolatility/Log/{exp_id}.fitting/fits")
results<-readRDS(file.path(fit_path, glue("fit_{config_id}.rds")))
rough_stochvol.nig<-results$fit[[run_id+1]]$fit



config_id<-115
run_id=0
fit_path=glue("/home/jianj0c/project/STP/notebook_SPDE/stochasticVolatility/Log/{exp_id}.fitting/fits")
results<-readRDS(file.path(fit_path, glue("fit_{config_id}.rds")))
rough_stochvolln<-results$fit[[run_id+1]]$fit



config_id<-121
run_id=0
fit_path=glue("/home/jianj0c/project/STP/notebook_SPDE/stochasticVolatility/Log/{exp_id}.fitting/fits")
results<-readRDS(file.path(fit_path, glue("fit_{config_id}.rds")))
rs_stochvol<-results$fit[[run_id+1]]$fit

In [30]:
models=list(rough_stochvol,rough_stochvol.t,rough_stochvol.nig,rough_stochvolln,rs_stochvol)

In [31]:
res_year <- cross_validation(models,train_test_indexes = build_index(length(date_timeindex), 252))
res_month <- cross_validation(models,train_test_indexes = build_index(length(date_timeindex), 22,size=30))
res_week <- cross_validation(models,train_test_indexes = build_index(length(date_timeindex), 5,size=30))

Fold: 1 / 3 
Model: Model 1 
Generating samples...

 *** inla.core.safe:  The inla program failed, but will rerun in case better initial values may help. try=1/1 

 *** inla.core.safe:  rerun with improved initial values 
Samples generated!
MSE: 0.000131739710962798 
DSS: -7.94008162986613 
CRPS: -0.00608109238137303 
SCRPS: -1.20393045950211 
Fold: 1 / 3 
Model: Model 2 
Generating samples...
Samples generated!
MSE: 0.00013294968026764 
DSS: -7.76173110554456 
CRPS: -0.00614612776304043 
SCRPS: -1.1825299891309 
Fold: 1 / 3 
Model: Model 3 
Generating samples...
Samples generated!
MSE: 0.000131858307680951 
DSS: -7.80911956593621 
CRPS: -0.00609529066507541 
SCRPS: -1.19446079878679 
Fold: 1 / 3 
Model: Model 4 
Generating samples...
Samples generated!
MSE: 0.0010631393677301 
DSS: -0.00844021205746028 
CRPS: -0.227286027230768 
SCRPS: 0.769303031830485 
Fold: 1 / 3 
Model: Model 5 
Generating samples...

 *** inla.core.safe:  The inla program failed, but will rerun in case better ini

In [33]:

library(xtable)


In [34]:

for( result_df in list(res_year,res_month,res_week) ){
    xtable(result_df, caption = "Example Data Frame", label = "tab:example_df",digits=c(0,0,4,4,4,4))
    # Convert to LaTeX table
    latex_table <- xtable(result_df, caption = "Example Data Frame", label = "tab:example_df",igits=10)

    # Print the LaTeX table
    print.xtable(latex_table, float.placement = "H")

}

% latex table generated in R 4.3.3 by xtable 1.8-4 package
% Tue Apr 23 10:27:17 2024
\begin{table}[ht]
\centering
\begin{tabular}{rlllll}
  \hline
 & Model & dss & mse & crps & scrps \\ 
  \hline
1 & Model 1 & -7.93027707746269 & 0.000131203967160899 & -0.00617910421176564 & -1.19587953113281 \\ 
  2 & Model 2 & -7.7683146339758 & 0.000132004302642555 & -0.00618095054278748 & -1.18269700204787 \\ 
  3 & Model 3 & -7.84547899839904 & 0.000130634090478235 & -0.00615353891463189 & -1.19241258626146 \\ 
  4 & Model 4 & 0.00410032668445688 & 0.000734289210041716 & -0.227063209477143 & 0.764751439313549 \\ 
  5 & Model 5 & -7.89911164030639 & 0.00013099819668956 & -0.00616341940145959 & -1.19471087117041 \\ 
   & Best & Model 1 & Model 3 & Model 4 & Model 1 \\ 
   \hline
\end{tabular}
\caption{Example Data Frame} 
\label{tab:example_df}
\end{table}
% latex table generated in R 4.3.3 by xtable 1.8-4 package
% Tue Apr 23 10:27:17 2024
\begin{table}[ht]
\centering
\begin{tabular}{rlllll}
  \hl

In [22]:

for( result_df in list(res_year,res_month,res_week) ){
    xtable(result_df, caption = "Example Data Frame", label = "tab:example_df",digits=c(0,0,4,4,4,4))
    # Convert to LaTeX table
    latex_table <- xtable(result_df, caption = "Example Data Frame", label = "tab:example_df",igits=10)

    # Print the LaTeX table
    print.xtable(latex_table, float.placement = "H")

}

% latex table generated in R 4.3.3 by xtable 1.8-4 package
% Mon Apr 22 16:11:43 2024
\begin{table}[ht]
\centering
\begin{tabular}{rlllll}
  \hline
 & Model & dss & mse & crps & scrps \\ 
  \hline
1 & Model 1 & -7.20667218733098 & 0.000267488572713831 & -0.00851386287215163 & -1.03469416246952 \\ 
  2 & Model 2 & -7.07278636254917 & 0.000267205503994898 & -0.00856452340853499 & -1.01956452658355 \\ 
  3 & Model 3 & -7.07343807624542 & 0.00026654593209653 & -0.00853378743635089 & -1.02414498154657 \\ 
  4 & Model 4 & -0.000390336015960651 & 0.00106608230434287 & -0.228554367456972 & 0.763468494826343 \\ 
  5 & Model 5 & -7.20936850959021 & 0.000267213257044212 & -0.0085005351017717 & -1.0359025000778 \\ 
   & Best & Model 5 & Model 3 & Model 4 & Model 5 \\ 
   \hline
\end{tabular}
\caption{Example Data Frame} 
\label{tab:example_df}
\end{table}
% latex table generated in R 4.3.3 by xtable 1.8-4 package
% Mon Apr 22 16:11:43 2024
\begin{table}[ht]
\centering
\begin{tabular}{rlllll}
  \hl

In [18]:
result_df

[1] "Intercept"